In [1]:
from openai import OpenAI, AsyncOpenAI
import asyncio
import os
from utils import get_dataset,get_ans_words_chard
from datasets import load_dataset
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))


from tqdm import tqdm   
import emoji
import json


In [2]:
def save_results(temp, file_name):

    ## Initialize the file if it does not exist
    if not os.path.exists(file_name):
        with open(file_name,'w') as file:
            json.dump([],file)
    
    file_data = []
    with open(file_name,'r') as file:
        file_data = json.load(file)

    file_data.extend(temp)
    ## sort the data by idx
    file_data = sorted(file_data, key=lambda d: d['idx'])

    with open(file_name,'w') as file:
        # file.seek(0)
        json.dump(file_data,file)
        

In [3]:
from utils import get_dataset,generate_prompt
import prompts

import numpy as np

errors = 0

model_name = 'gpt-3.5-turbo'
dataset_name = 'boda/guardian_naive_random'
# dataset_name = 'boda/guardian_word_initial_disjoint'
chatgpt_outputs_file = f"results/chatgpt_outputs/{model_name}_{dataset_name.split('/')[-1]}_all_inclusive.json"
# base_prompt ='LLAMA3_BASE_PROMPT'
base_prompt ='ALL_INCLUSIVE_PROMPT'

shots = 0
temperature = 0
dataset = get_dataset(dataset_name,
                split='test',
                prompt_key='prompt',
                prompt_head=base_prompt,
                shots=0,
                )

------------------Loading boda/guardian_naive_random/test, and using ALL_INCLUSIVE_PROMPT, and 0 shot prompt ------------------


Map:   0%|          | 0/28476 [00:00<?, ? examples/s]

In [4]:
dataset

Dataset({
    features: ['input', 'target', 'prompt'],
    num_rows: 28476
})

In [5]:

num_examples = len(dataset)
# num_examples = 10000

save_temps = []
done_ids = []
## check if there's a file already 
if os.path.exists(chatgpt_outputs_file):
    with open(chatgpt_outputs_file,'r') as file:
        done = json.load(file)
        done_ids = list(set([d['idx'] for d in done]))
    print(f"Resuming from {len(done_ids)} examples")
    if len(done_ids) >= num_examples:
        print("Already done")
        exit(0)
else:
   print(f'Begining a new generation')


Resuming from 15201 examples


In [6]:
# done = [x for x in done if 'response' in x.keys()]

# print(f"Already done {len(done)} examples")

In [7]:
# s = set()
# new_done = []
# dups = 0
# for i in done:
#     if i['idx'] in s:
#         dups += 1
#         # print(i['idx'])
#         # break
#     else:
#         new_done.append(i)

#     s.add(i['idx'])

# print(f"Already done {len(s)} examples, {dups} duplicates, unique{len(new_done)}")

In [8]:
# write = sorted(new_done, key=lambda d: d['idx'])

# print(len(write))

# with open('results/chatgpt_outputs/gpt-3.5-turbo_guardian_naive_random_no-sampling_temp-zero.json','w') as file:
#     # file.seek(0)
#     json.dump(write,file)
    

In [9]:





# with open(chatgpt_outputs_file, 'a') as f:
for idx ,sample in enumerate(tqdm(dataset.select(range(num_examples)))):
    
    # if this example is done, then continue
    if idx in done_ids:
        continue
    
    clue = sample['input']
    target = sample["target"]
    prompt = sample['prompt']
    
    try:
      # correct_answers.append(clue["target"])
      clue_message = {"role": "user", "content": prompt }#clue['prompt']}
      completion = client.chat.completions.create(
        model=model_name,
        messages=[
          clue_message
        ],
        temperature=temperature
      )

      response = completion.choices[0].message.content.lower()
      for l in response.split('\n'):
        if 'answer:' in l:
          response = l.split('answer:')[1].strip().replace(',','').replace('.','').replace('?','').replace('!','').strip()
      save_temps.append({'idx': idx, 'clue': clue,'response': response, 'target': target})
      done_ids.append(idx)

    except:
      # save_temps.append({'idx': idx})
      errors += 1
      
    if idx % 100 == 0 or idx == num_examples - 1:
      save_results(save_temps,chatgpt_outputs_file)
      save_temps = []


        

    


100%|██████████| 28476/28476 [9:34:28<00:00,  1.21s/it]   


In [10]:
import json
with open(chatgpt_outputs_file) as f:
    d = json.load(f)


chatgpt_outputs = []
correct_answers = []
clues = []
errors = 0
for i in d:
    if 'response' in i:
        chatgpt_outputs.append(i['response'])
        correct_answers.append(i['target'])
        clues.append(i['clue'])
    else:
        errors += 1

assert len(chatgpt_outputs) == len(correct_answers)


In [11]:
from calc_scores import calc_and_save_acc
from utils import crop_predictions
import pandas as pd

data_args = pd.DataFrame({
    'dataset': dataset_name,
    'split': 'test',
    'prompt_key': 'prompt',
    'prompt_head': base_prompt,
    'n_shots': shots,
},index=[0])

model_args = pd.DataFrame({
    'model_name_or_path': model_name,
},index=[0])

cleaned_outputs = crop_predictions(clues, chatgpt_outputs)

print(clues,chatgpt_outputs, cleaned_outputs)
calc_and_save_acc(
                chatgpt_outputs, 
                correct_answers, 
                cleaned_predictions= cleaned_outputs, 
                save_file = chatgpt_outputs_file.replace('.json','_results.txt'), 
                write_outputs = True,
                model_args = model_args,
                data_args= data_args,)

['Achy shaking stopped by iodine, salt and kaolin (5,4)', 'Paid the bail for a sound woman (5)', 'In Germany, thin girls can eat this! (8)', 'Old official partial to free verse (5)', "Coarse-sounding son's in the wilderness (7)", 'Turns up in chilly surroundings yet well received (7)', 'Foreign currency racket completely disheartened auditor (5)', 'The fellow apprehending one felon (5)', 'Dames popular with sailors? (6)', "Last year's leaders entertained by regular journalist (6)", 'Butcher breaks bent pan (7)', 'Improved a carving with direction (4,2)', '\'s "a hairy man", speaking in child\'s play (6)', 'Bend down to king in Chesterfield (6)', 'Message left in note, eg "Castle in the morning" (8)', "A weightwatchers' congress? (4)", 'Dim clue reset by compiler finally for "cimbalom" (8)', 'In the chair, restraining Democrat under influence of drugs (7)', 'Horny beast imagined finding garment ripped, models having fled (7)', 'Sword giving enemy cut to one leg (4)', 'Very cutting, but 